In [146]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')
import spacy
from spacy.language import Language
from spacy import displacy
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [207]:
@Language.factory('make_street_entity_matcher')
def make_street_entity_matcher(nlp, name, file_name:str, entity_name:str):
    return EntityMatcher(nlp, file_name, entity_name)

ValueError: [E004] Can't set up pipeline component: a factory for 'make_street_entity_matcher' already exists. Existing factory: <function make_street_entity_matcher at 0x149c5c620>. New factory: <function make_street_entity_matcher at 0x149d06378>

In [235]:
class EntityMatcher:
    def __init__(self, nlp, file_name, entity_name):
        line_list = open(file_name,'r').readlines()
        phrase_list = []
        for l in line_list:
            phrase_list.append(l.strip())
        phrase_patterns = [nlp(text) for text in phrase_list]
        print(phrase_patterns)
        self.matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
        self.matcher.add(entity_name, None, *phrase_patterns)

    def __call__(self, doc):
        matches = self.matcher(doc)
        seen_tokens = set()
        new_entities = []
        entities = doc.ents
        for match_id, start, end in matches:
            if start not in seen_tokens and end - 1 not in seen_tokens:
                new_entities.append(Span(doc, start, end, label=match_id))
                entities = [
                    e for e in entities if not (e.start < end and e.end > start)
                ]
                seen_tokens.update(range(start, end))

        doc.ents = tuple(entities) + tuple(new_entities)
        return doc

In [236]:
#nlp = spacy.load("nl_core_news_sm")
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("make_street_entity_matcher", config={"file_name":'../../Data/street_names_north_holland.txt',"entity_name": "STREET"}, after="ner")

<function make_street_entity_matcher at 0x149c5c620>
[A A de Lannoy-Willemsstr, A Eduardine Kokplnts, A. Reitsmakade, A.A.H. Struijckenkade, A.J. Koejemansstraat, A.J. Krophollerstraat, A.M. de Jongstraat, A.S. Onderwijzerhof, A.W. Grootehof, Aaf Bouberstraat, Aagtdorperpad, Aakstraat, Aalbersestraat, Aalsmeerplein, Aalsmeerweg, Aambeeldstraat, Aandewind, Aardbeistraat, Aart van der Leeuwstraat, Abbenesstraat, Abberdaan, Abcouderstraatweg, Abebe Bikilalaan, Abraham de Haanstraat, Abraham Dudok v Heelstr, Abraham Kuyperplein, Abraham Staalmanplein, Abraham van der Hartstr, Abrikozenstraat, Accraweg, Accumulatorweg, Achillesstraat, Achter Oosteinde, Achtergouwtje, Achtergracht, Achtergracht AB, Achterlaan, Achtersteven, Adam van Germezhof, Adamello, Adelaarsweg, Adele Opzoomerstraat, Adingerdorphof, Admiraal de Ruijterweg, Admiraal Helfrichstraat, Admiralengracht, Admiraliteitstraat, Adolf van Nassaustraat, Adriaan Dorsmanstraat, Adriaan Kluitstraat, Adriaan Loosjesstraat, Adriaan van Be

In [237]:
file = '../../Data/AH/Haarlemmerdijk 1.txt'
tokenisedList = []
with open(file,'r') as f:
    line = f.readline()
    while line:
        line = f.readline()
        nlpLine = nlp(line)
        tokenisedList.append(nlpLine)

In [ ]:
# if you want to keep specific stop words
# nlp.Defaults.stop_words.remove()
# nlp.vocab['beyond'].is_stop = False

In [238]:
displacy.render(tokenisedList, style='ent', jupyter=True)
displacy.render(tokenisedList, style='ent', options={'distance':50})